# YOLOv11 LGG MRI Tumor Segmentation
**Just upload your kaggle.json when asked**

In [ ]:
!pip install -q torch torchvision ultralytics opencv-python kaggle tqdm ipython

from IPython.display import FileLink, display
from google.colab import files
import os, cv2, numpy as np, random
from pathlib import Path
from tqdm import tqdm
from ultralytics import YOLO

# Upload kaggle.json
print("Upload your kaggle.json (Kaggle → Account → Create New Token)")
uploaded = files.upload()
!mkdir -p ~/.kaggle && cp kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

# Download dataset
!kaggle datasets download -d mateuszbuda/lgg-mri-segmentation
!unzip -qo lgg-mri-segmentation.zip -d data

# Convert to YOLO format
raw = Path("data/lgg-mri-segmentation")
yolo = Path("yolo_dataset")
for p in ["images/train","images/val","labels/train","labels/val"]: (yolo/p).mkdir(parents=True, exist_ok=True)

pairs = []
for patient in raw.iterdir():
    if patient.is_dir():
        imgs = list(patient.glob("*[0-9].tif"))
        for img in imgs:
            mask = patient / f"{img.stem}_mask.tif"
            if mask.exists(): pairs.append((img, mask))

random.seed(42)
random.shuffle(pairs)
split = int(0.8 * len(pairs))

def convert(img_path, mask_path, split):
    img = cv2.imread(str(img_path))
    mask = cv2.imread(str(mask_path), 0)
    h, w = img.shape[:2]
    name = f"{img_path.parent.name}_{img_path.name}"
    cv2.imwrite(str(yolo/f"images/{split}/{name}"), img)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    segs = []
    for cnt in contours:
        if cv2.contourArea(cnt) > 30:
            c = cnt.flatten().astype(float)
            c[0::2] /= w
            c[1::2] /= h
            segs.append(f"0 {' '.join(map(str, c))}")
    if segs:
        with open(yolo/f"labels/{split}/{name.replace('.tif','.txt')}", "w") as f:
            f.write("\n".join(segs))

for p in tqdm(pairs[:split], desc="Train"): convert(p[0], p[1], "train")
for p in tqdm(pairs[split:], desc="Val"):   convert(p[0], p[1], "val")

# data.yaml
with open("data.yaml","w") as f:
    f.write("""path: yolo_dataset
train: images/train
val: images/val
nc: 1
names: ['tumor']""")

# Train
model = YOLO("yolov11s-seg.pt")
model.train(data="data.yaml", epochs=120, imgsz=256, batch=32, device=0,
            project="runs", name="lgg", exist_ok=True, patience=30, close_mosaic=10, freeze=10)

# Demo video
best = YOLO("runs/lgg/weights/best.pt")
val_imgs = sorted(list(Path("yolo_dataset/images/val").glob("*.tif")))[:30]
frames = []
for p in tqdm(val_imgs, desc="Video"):
    img = cv2.imread(str(p))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    res = best(p, conf=0.25, verbose=False)[0]
    overlay = img_rgb.copy()
    if res.masks is not None:
        for mask, box in zip(res.masks.data.cpu(), res.boxes.data.cpu()):
            m = cv2.resize(mask.numpy(), (img_rgb.shape[1], img_rgb.shape[0])) > 0.5
            overlay[m] = overlay[m] * 0.6 + np.array([0,191,255]) * 0.4
            cv2.putText(overlay, f"Tumor {box[4]:.2f}", (50,80), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 4)
    cv2.putText(overlay, "YOLOv11-seg LGG MRI | Omid Sakaki", (20,img_rgb.shape[0]-20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,255), 2)
    frames.append(overlay)

out = cv2.VideoWriter("Demo.mp4", cv2.VideoWriter_fourcc(*"mp4v"), 6, (img_rgb.shape[1], img_rgb.shape[0]))
for f in frames: out.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
out.release()

display(FileLink("Demo.mp4"))
print("All done! Click the link above to download your video")